In [ ]:
from sklearn import tree
import os # For file reading
import json # For json reading

In [ ]:
#clf = classifier
clf = tree.DecisionTreeClassifier()

RELEVANT_STRING = 'relevant'
NOT_RELEVANT_STRING = 'not relevant'

X = []
Y = []


relevant_dir = os.path.abspath(os.curdir) + '/relevant/'
not_relevant_dir = os.path.abspath(os.curdir) + '/not_relevant/'
file_dir = os.path.abspath(os.curdir) + '/files/'


# Loop through all relevant
for filename in os.listdir(relevant_dir):
    
    # Skip directories
    if os.path.isdir(os.path.join(relevant_dir, filename)):
        continue
    
    # Open every file
    with open(relevant_dir + filename, 'r') as file:
        file_data = json.loads(file.read())
        
        X.append(list(file_data['word_counts'].values()) + [len(file_data['content']), file_data['order_id']])
#         X.append([len(file_data['content'])])
        Y.append(RELEVANT_STRING)
    
        
# Loop through all irrelevant
for filename in os.listdir(not_relevant_dir):
    
    # Skip directories
    if os.path.isdir(os.path.join(not_relevant_dir, filename)):
        continue
        
    
    # Open every file
    with open(not_relevant_dir + filename, 'r') as file:
        file_data = json.loads(file.read())
        
        X.append(list(file_data['word_counts'].values()) + [len(file_data['content']), file_data['order_id']])
#         X.append([len(file_data['content'])])
        Y.append(NOT_RELEVANT_STRING)
            

clf = clf.fit(X, Y)

relevant = 0
irrelevant = 0

for filename in os.listdir(file_dir):
    


    # Skip directories
    if os.path.isdir(os.path.join(file_dir, filename)):
        continue
        
    # Skip test set files
    if os.path.exists(os.path.join(relevant_dir, filename)) or os.path.exists(os.path.join(not_relevant_dir, filename)):
        continue

    
    # Open every file
    with open(file_dir + filename, 'r') as file:
        
        file_data = json.loads(file.read())
        
        prediction = clf.predict(
                [list(file_data['word_counts'].values()) + [len(file_data['content']), file_data['order_id']]]
        )
        
        if prediction[0] == RELEVANT_STRING:
            relevant += 1
        else:
            irrelevant += 1
        

print("Concluded analysis:")
print(str(relevant) + " relevant files found.")
print(str(irrelevant) + " irrelevant files found.")